# 02: Quantitative Mechanical Phenotyping & Feature Extraction

## 🔬 Overview
This notebook performs high-throughput feature extraction from segmented cell masks. It quantifies the mechanical impact of gene ME480 on cancer cell populations by calculating deformation, circularity, and solidity. 

### Quality Control Filters:
1. **Area Distribution Filter:** Excludes outliers using a **90% Confidence Interval (CI)** to ensure analysis focuses on healthy, non-clumped cell populations.
2. **Aspect Ratio Filter:** Excludes elongated debris or cell doublets (valid range: 0.5 - 2.0).

In [ ]:
import os
import numpy as np
import pandas as pd
import math
import seaborn as sns
import matplotlib.pyplot as plt
from skimage.measure import regionprops, label
from skimage.io import imread

# =============================================================================
# 1. DEFINE PATHS & CONDITIONS
# =============================================================================
base_dir = os.path.dirname(os.getcwd())
mask_root = os.path.join(base_dir, "data", "processed", "masks")
output_csv = os.path.join(base_dir, "data", "results", "ME480_Mechanical_Features.csv")

os.makedirs(os.path.dirname(output_csv), exist_ok=True)

# Define conditions based on your folder names
conditions = ["Native_cells", "OE_cells", "KD_cells"]
all_data = []
area_samples = []

### Step 1: Population Sampling for Outlier Removal
We sample all detected objects to establish the global 90% Confidence Interval for cell area.

In [ ]:
for condition in conditions:
    cond_path = os.path.join(mask_root, condition)
    if not os.path.exists(cond_path): continue
    
    for mask_file in os.listdir(cond_path):
        if mask_file.endswith('.png'):
            img = imread(os.path.join(cond_path, mask_file))
            labeled = label(img > 0)
            for prop in regionprops(labeled):
                area_samples.append(prop.area)

# Calculate 90% CI bounds (Z = 1.645)
mean_area = np.mean(area_samples)
std_area = np.std(area_samples)
lower_bound = mean_area - 1.645 * std_area
upper_bound = mean_area + 1.645 * std_area

print(f"90% CI for Area Filter: [{lower_bound:.2f}, {upper_bound:.2f}] pixels")

# Plotting Distribution
plt.figure(figsize=(8, 5))
sns.histplot(area_samples, kde=True, color='teal')
plt.axvline(lower_bound, color='red', linestyle='--')
plt.axvline(upper_bound, color='red', linestyle='--')
plt.title("Global Cell Area Distribution (with 90% CI Bounds)")
plt.show()

### Step 2: Extraction Loop with Filter Integration
We now apply the calculated area filter and the bounding box aspect ratio filter.

In [ ]:
for condition in conditions:
    cond_path = os.path.join(mask_root, condition)
    if not os.path.exists(cond_path): continue
    
    files = sorted([f for f in os.listdir(cond_path) if f.endswith('.png')])
    
    for frame_idx, mask_file in enumerate(files):
        img = imread(os.path.join(cond_path, mask_file))
        labeled = label(img > 0)
        
        for prop in regionprops(labeled):
            # Filter 1: Area CI
            if prop.area < lower_bound or prop.area > upper_bound: continue
            
            # Filter 2: Bounding Box Aspect Ratio
            min_r, min_c, max_r, max_c = prop.bbox
            aspect_ratio = (max_c - min_c) / (max_r - min_r) if (max_r - min_r) > 0 else 0
            if aspect_ratio < 0.5 or aspect_ratio > 2.0: continue
            
            # Feature Extraction
            peri = prop.perimeter
            # Deformation formula as defined in RT-DC literature
            defo = 1 - (2 * math.sqrt(math.pi * prop.area)) / peri if peri > 0 else 0
            
            all_data.append({
                "Condition": condition,
                "Frame": frame_idx,
                "Area": prop.area,
                "Deformation": defo,
                "Solidity": prop.solidity,
                "Circularity": (4 * np.pi * prop.area) / (peri**2) if peri > 0 else 0,
                "MajorAxis": prop.major_axis_length,
                "Centroid_X": prop.centroid[1],
                "Centroid_Y": prop.centroid[0]
            })

df_results = pd.DataFrame(all_data)
df_results.to_csv(output_csv, index=False)
print(f"Feature extraction complete. Total cells analyzed: {len(df_results)}")

### Step 3: Comparative Analysis Visualization

In [ ]:
plt.figure(figsize=(10, 6))
sns.scatterplot(data=df_results, x="Area", y="Deformation", hue="Condition", alpha=0.5)
plt.title("Cell Deformation vs Area Across Genotypes", fontsize=14)
plt.grid(alpha=0.2)
plt.show()

plt.figure(figsize=(8, 5))
sns.boxplot(data=df_results, x="Condition", y="Deformation", palette="Set2")
plt.title("Mechanical Phenotype Comparison (ME480 Impact)")
plt.show()